In [1]:
import numpy as np
import intracluster_smote
import evaluation
import matplotlib.pyplot as plt
%matplotlib inline
import mnist_utils
import asirra_utils
import pandas as pd
from IPython.display import display
import seaborn as sns
import sklearn
import sklearn.cluster
import sklearn.model_selection
import traceback
import random
from datetime import datetime, timedelta
import os
import keras
import autoencoder_keras
from keras.callbacks import TensorBoard
# Fixup for keras for Tensorboard > 0.12
import tensorflow as tf
tf.merge_all_summaries = tf.summary.merge_all
tf.train.SummaryWriter = tf.summary.FileWriter

Using TensorFlow backend.


In [2]:
# Import Credit Card Dataset
# https://www.kaggle.com/dalpozz/creditcardfraud
cc_scaler = sklearn.preprocessing.MinMaxScaler()
def load_cc(ratio=None):
    cc_data_raw = pd.read_csv('datasets/creditcard.csv')
    cc_data_target = cc_data_raw['Class']
    cc_data = cc_data_raw.drop(['Class'], axis=1)
    cc_data = cc_scaler.fit_transform(cc_data)
    return np.asarray(cc_data), np.asarray(cc_data_target)
cc_data_full = load_cc()
train_id,test_id = next(sklearn.model_selection.StratifiedShuffleSplit(test_size=0.5, n_splits=1).split(cc_data_full[0],cc_data_full[1]))
cc_data_train = cc_data_full[0][train_id]
cc_data_train_target = cc_data_full[1][train_id]
cc_data_test = cc_data_full[0][test_id]
cc_data_test_target = cc_data_full[1][test_id]

In [3]:
# try different architectures of ae for reconstruction error with training_set/validation_set
# try different #layers
# try different #units
# verify: does linear transformation (i.e. PCA) affect the performance of the neural network?
# check reconstruction quality on an individual sample level - how well does it actually reconstruct individual points?
# different optimizers
# initialization of weights: Xavier Initialization (normal distr. with SD depending on # of input units)
# activation functions: try tanh, relu
# architecture of network for this dataset kaggle
# try not reducing (i.e. compressing) at all

In [31]:
default_params = {
    'layers': [30,20,15,20,30],
    'optimizer': 'adadelta',
    'loss': 'binary_crossentropy',
    'regularize': False,
    'batch_size':256,
    'epoch_count':5000,
    'name':'default 30-20-15',
}
grid = [default_params,
 ###### LAYER STRUCTURE ########
{
    'layers': [30,15,15,10,15,15,30],
    'name':'L 30-15-15-10',
},{
    'layers': [30,15,30],
    'name':'L 30-15',
},{
    'layers': [30,15,10,15,30],
    'name':'L 30-15-10',
},{
    'layers': [30,30,30],
    'name':'L 30-30',
},###### OPTIMIZERS ########
{
    'optimizer': keras.optimizers.SGD(lr=0.01, momentum=0.7, decay=0.0, nesterov=True),
    'name':'O SGD Nesterov 0.7',
},{
    'optimizer': 'adagrad',
    'name':'O Adagrad',
},{
    'optimizer': 'adagrad',
    'name':'O Nadam',
},##### BATCH SIZE #####
{
    'batch_size':1000,
    'name':'B 1,000',
},{
    'batch_size':10000,
    'name':'B 10,000',
},#### REGULIZATION #####
# {
#     'regularize':True,
#     'name':'Regularized',
# },##### COMBINATION ######
{
    'optimizer': 'adagrad',
    'batch_size':1000,
    'layers': [30,20,10,20,30],
    'epoch_count':1500,
    'name':'O Adagrad; B 1,000; L 30-20-10',
},{
    'optimizer': 'adagrad',
    'batch_size':1000,
    'layers': [30,15,10,15,30],
    'epoch_count':1500,
    'name':'O Adagrad; B 1,000; L 30-15-10',
},{
    'optimizer': 'adagrad',
    'batch_size':1000,
    'layers': [30,10,30],
    'epoch_count':1500,
    'name':'O Adagrad; B 1,000; L 30-10',
},
]

In [33]:
for i, configuration in enumerate(grid):
    if i < 12: continue
    params = {**default_params, **configuration}
    print('\n',params)
    folder = 'results/cc_optimization_1500/{0} {1}'.format(i, params['name'])
    ae = autoencoder_keras.Autoencoder(
        layers=params['layers'],
        regularize=params['regularize'],
        optimizer=params['optimizer'],
        loss=params['loss'],
        training_set=cc_data_train
    )
    ae.fit(cc_data_train, cc_data_train,
            nb_epoch=params['epoch_count'],
            batch_size=params['batch_size'],
            shuffle=True,
            verbose=0,
            callbacks=[TensorBoard(log_dir=(folder))],
            validation_data=(cc_data_test,cc_data_test)
    )
    try:
        np.save(folder+'/ae_weights', ae.get_parameters())
    except: pass


 {'optimizer': 'adagrad', 'batch_size': 1000, 'epoch_count': 1500, 'loss': 'binary_crossentropy', 'name': 'O Adagrad; B 1,000; L 30-15-10', 'layers': [30, 10, 30], 'regularize': False}


In [30]:
i=11
params = {**default_params, **grid[i]}
ae = autoencoder_keras.Autoencoder(
        layers=params['layers'],
        regularize=params['regularize'],
        optimizer=params['optimizer'],
        loss=params['loss'],
        training_set=cc_data_train
    )
folder = 'results/cc_optimization_5000/{0} {1}'.format(i, params['name'])
ae.set_parameters(np.load(folder+'/ae_weights.npy'))
print(folder+'/ae_weights.npy')

IndexError: list index out of range

In [ ]:
ff = ae.decode(ae.encode(cc_data_test))
diff = cc_data_test - ff
diff_at_target = diff[np.asarray(cc_data_test_target, dtype='bool')]

In [ ]:
display(pd.DataFrame(diff.reshape((diff.size))).describe())
sns.heatmap(diff[np.random.choice(diff.shape[0], size=30)])

In [ ]:
display(pd.DataFrame(diff_at_target.reshape((diff_at_target.size))).describe())
sns.heatmap(diff_at_target[np.random.choice(diff_at_target.shape[0], size=30)])